In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
!wget -nc https://raw.githubusercontent.com/ceos-seo/odc-colab/master/odc_colab.py && git clone https://github.com/ceos-seo/odc-colab
from odc_colab import odc_colab_init
odc_colab_init(install_odc_gee=True)

In [ ]:
from odc_colab import populate_db
populate_db()

In [ ]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert.preprocessors import CellExecutionError
from nbclient.exceptions import CellControlSignal

## Get a list of notebooks to run

In [ ]:
import sys
import os
nb_path = '/content/odc-colab/notebooks/'

excluded_notebooks = {'01.01.Getting_Started_ODC_and_Colab.ipynb'}

def ls_nb(nb_path):
  nb_files = []
  for nb_path, dirs, files in os.walk(nb_path):
    files = set(files).difference(excluded_notebooks)
    nb_files += [os.path.join(nb_path, f) for f in files if '.ipynb' in f]
  return sorted(nb_files)

In [ ]:
notebook_file_paths = ls_nb(nb_path)

## Run the notebooks and record their status (e.g. working, error) to HTML as each completes

In [ ]:
import re
import pandas
ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
full_report = pandas.DataFrame(columns=['Notebook', 'Status', 'Bug Description'])
error_report = pandas.DataFrame(columns=['Notebook', 'Status', 'Bug Description'])
success_report = pandas.DataFrame(columns=['Notebook', 'Status', 'Bug Description'])
# Unless this cell is rerun, only run notebooks that have not been run successfully yet.
success_notebooks = []

**To rerun all notebooks, the file `success_nbks_file_name` must be deleted and then this notebook must be restarted**

In [ ]:
import pickle

# Load the list of successful notebooks.
success_nbks_file_name = 'success_nbks.pkl'
if os.path.exists(success_nbks_file_name):
    success_nbks_file_in = open(success_nbks_file_name, 'rb')
    success_notebooks = pickle.load(success_nbks_file_in)

notebook_file_paths_to_run = [nbk_pth for nbk_pth in notebook_file_paths if nbk_pth not in success_notebooks]
for notebook_file_path in notebook_file_paths_to_run:
    print(f'Running {notebook_file_path}')
    run_result = {'Notebook': notebook_file_path, 'Status': 'Working', 'Bug Description': ''}
    with open(notebook_file_path, 'r+', encoding='utf-8') as notebook_file:
        notebook = nbformat.read(notebook_file, as_version=4)
        notebook_runner = ExecutePreprocessor(timeout=None)
        try:
            notebook_runner.preprocess(notebook, {'metadata': {'path': '/content'}})
        except CellExecutionError as err:
            run_result['Status'] = 'Error'
            run_result['Bug Description'] = err
        error = run_result['Status'] == 'Error'
        # Save the notebook.
#         nbformat.write(notebook, notebook_file_path)
    full_report = full_report.append(run_result, ignore_index=True)
    if error:
        error_report = error_report.append(run_result, ignore_index=True)
    else:
        # Record that this notebook ran successfully to avoid running it again for this testing session.
        success_notebooks.append(notebook_file_path)
        success_report = success_report.append(run_result, ignore_index=True)
        success_nbks_file_out = open(success_nbks_file_name, 'wb') 
        pickle.dump(success_notebooks, success_nbks_file_out)
    full_report.to_html('full_test_report.html', escape=False, formatters={'Bug Description': lambda e: ansi_escape.sub('', str(e).replace('\n', '<br/>'))})
    error_report.to_html('error_report.html', escape=False, formatters={'Bug Description': lambda e: ansi_escape.sub('', str(e).replace('\n', '<br/>'))})
    success_report.to_html('success_report.html', escape=False, formatters={'Bug Description': lambda e: ansi_escape.sub('', str(e).replace('\n', '<br/>'))})

Running /content/odc-colab/notebooks/02.01.Colab_Cloud_Statistics_L8.ipynb
Running /content/odc-colab/notebooks/02.02.Colab_Cloud_Statistics_S2.ipynb
Running /content/odc-colab/notebooks/02.03.Colab_Median_Mosaic_L8.ipynb
Running /content/odc-colab/notebooks/02.04.Colab_Water_WOFS_L8.ipynb
Running /content/odc-colab/notebooks/02.05.Colab_Spectral_Products_L8.ipynb
Running /content/odc-colab/notebooks/02.06.Colab_Vegetation_Change_L8.ipynb
Running /content/odc-colab/notebooks/02.07.Colab_Vegetation_Phenology_L8.ipynb
Running /content/odc-colab/notebooks/02.08.01.Colab_S1_Data_Viewer.ipynb
Running /content/odc-colab/notebooks/02.08.02.Colab_S1_Flooding.ipynb
Running /content/odc-colab/notebooks/02.09.Colab_Mission_Coincidences.ipynb
Running /content/odc-colab/notebooks/02.10.Colab_VIIRS_Night_Lights.ipynb


### Export the results to a CSV

In [ ]:
full_report['Bug Description'] = full_report['Bug Description'].map(lambda e: ansi_escape.sub('', str(e)))
full_report.to_csv('full_test_report.csv')
error_report['Bug Description'] = error_report['Bug Description'].map(lambda e: ansi_escape.sub('', str(e)))
error_report.to_csv('error_report.csv')
success_report['Bug Description'] = success_report['Bug Description'].map(lambda e: ansi_escape.sub('', str(e)))
success_report.to_csv('success_report_report.csv')

# Display Results

In [ ]:
from IPython.display import HTML
with open('./full_test_report.html', 'r') as report:
  report = report.read()

In [ ]:
HTML(report)

,Notebook,Status,Bug Description
0,/content/odc-colab/notebooks/02.01.Colab_Cloud_Statistics_L8.ipynb,Working,
1,/content/odc-colab/notebooks/02.02.Colab_Cloud_Statistics_S2.ipynb,Working,
2,/content/odc-colab/notebooks/02.03.Colab_Median_Mosaic_L8.ipynb,Working,
3,/content/odc-colab/notebooks/02.04.Colab_Water_WOFS_L8.ipynb,Working,
4,/content/odc-colab/notebooks/02.05.Colab_Spectral_Products_L8.ipynb,Working,
5,/content/odc-colab/notebooks/02.06.Colab_Vegetation_Change_L8.ipynb,Working,
6,/content/odc-colab/notebooks/02.07.Colab_Vegetation_Phenology_L8.ipynb,Working,
7,/content/odc-colab/notebooks/02.08.01.Colab_S1_Data_Viewer.ipynb,Working,
8,/content/odc-colab/notebooks/02.08.02.Colab_S1_Flooding.ipynb,Working,
9,/content/odc-colab/notebooks/02.09.Colab_Mission_Coincidences.ipynb,Working,
